<a href="https://colab.research.google.com/github/Redislabs-Solution-Architects/financial-vss/blob/main/redisvl-02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG from scratch with RedisVL (Python)

![Redis](https://redis.com/wp-content/themes/wpx/assets/images/logo-redis.svg?auto=webp&quality=85,75&width=120)

This notebook uses [redisvl](https://redisvl.com), a dedicated Python client library for using Redis as a vector database, to perform document + embdding indexing, semantic search, and RAG with an LLM (from scratch).

## Setup and Data Prep

### Pull Github Materials
We need to clone the supporting materials from github.

In [ ]:
# This clones your git repository into a directory named 'temp_repo'.
!git clone https://github.com/Redislabs-Solution-Architects/financial-vss.git temp_repo

# This command moves the 'resources' directory from 'temp_repo' to your current directory.
!mv temp_repo/resources .
!mv temp_repo/requirements.txt .

# This deletes the 'temp_repo' directory, cleaning up the unwanted files.
!rm -rf temp_repo


### Install Python Dependencies

In [ ]:
!pip install -q -r requirements.txt

In [ ]:
import warnings

warnings.filterwarnings("ignore")

### Load and Extract PDF Docs

Now we will load a single financial (10k filings) doc and preprocess it using some LangChain helpers.


- `UnstructuredFileLoader` is not the only document loader type that LangChain provides. Docs: https://python.langchain.com/docs/integrations/document_loaders/unstructured_file
- `RecursiveCharacterTextSplitter` is what we use to create smaller chunks of text from the doc. Docs: https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter


In [ ]:
import os

# Load list of pdfs
data_path = "resources/"
docs = [os.path.join(data_path, file) for file in os.listdir(data_path)]

print("Listing available documents ...", docs)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader

# pick out the Nike doc for this exercise
doc = [doc for doc in docs if "nke" in doc][0]

loader = UnstructuredFileLoader(
    doc, mode="single", strategy="fast"
  )

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500, chunk_overlap=0
  )

chunks = loader.load_and_split(text_splitter)

print("Done preprocessing. Created", len(chunks), "chunks of the original pdf", doc)

### Create dense propositions from raw text

One technique we can use to improve the quality of retrieval is to leverage an LLM from OpenAI during ETL. We will prompt the LLM to summarize and decompose the raw pdf text into more discrete propositional phrases. This will enhance the clarity of the text and improve semantic retrieval for RAG.

The goal is to utilize a preprocessing technique similar to what's outlined here:
https://github.com/langchain-ai/langchain/blob/master/templates/propositional-retrieval/propositional_retrieval/proposal_chain.py

In [ ]:
import openai
import os
import getpass


CHAT_MODEL = "gpt-3.5-turbo-0125"


if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY")


In [ ]:
import tqdm
import json


def create_dense_props(chunk):
    """Create dense representation of raw text content."""
    # The system message here should be HEAVILY customized for your specific use case
    SYSTEM_PROMPT = """
    You are a helpful PDF extractor tool. You will be presented with segments from
    raw PDF documents composed of 10k SEC filings information about public companies.

    Decompose and summarize the raw content into clear and simple propositions,
    ensuring they are interpretable out of context. Consider the following rules:
    1. Split compound sentences into simpler dense phrases that retain existing
    meaning.
    2. Simplify technical jargon or wording if possible while retaining existing
    meaning.
    2. For any named entity that is accompanied by additional descriptive information,
    separate this information into its own distinct proposition.
    3. Decontextualize the proposition by adding necessary modifier to nouns or
    entire sentences and replacing pronouns (e.g., "it", "he", "she", "they", "this", "that")
    with the full name of the entities they refer to.
    4. Present the results as a list of strings, formatted in JSON, under the key "propositions".
    """

    response = openai.OpenAI().chat.completions.create(
        model=CHAT_MODEL,
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"Decompose this raw content using the rules above:\n{chunk.page_content} "}
        ]
    )
    res = response.choices[0].message.content
    return json.loads(res)["propositions"]



props = [
    create_dense_props(chunk) for chunk in tqdm.tqdm(chunks)
  ]

In [ ]:
# Compose chunks and generated propositions
print("Raw\n", chunks[8].page_content)
print("\nCleaned\n", props[8])

### Create text embeddings from "propositions"

In [ ]:
from redisvl.utils.vectorize import HFTextVectorizer

hf = HFTextVectorizer("sentence-transformers/all-MiniLM-L6-v2")

# Embed each set of propositions derived by the LLM
embeddings = hf.embed_many([" ".join(p) for p in props])

# Check to make sure we've created enough embeddings, 1 per document chunk
len(embeddings) == len(props) == len(chunks)

### Run Localized Redis Stack

If you don't have a remote Redis instance, use an in-notebook version of [Redis Stack](https://redis.io/docs/getting-started/install-stack/). Or you can provision your own free instance of [Redis Cloud](https://redis.com/try-free/).


Use the below code to download and run a localized version of Redis Stack here in the notebook.

In [ ]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

### Connect to Redis

By default this notebook would connect to the local instance of Redis Stack. If you have your own Redis Cloud instance - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [ ]:
# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# Construct URL
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

## Getting Started with RedisVL

### Create an index from schema
Below we connect to Redis and create an index for vector search that contains a single text field and vector field.

In [ ]:
from redis import Redis
from redisvl.schema import IndexSchema
from redisvl.index import SearchIndex

index_name = "redisvl"

schema = IndexSchema.from_dict({
  "index": {
    "name": index_name,
    "prefix": "chunk"
  },
  "fields": [
    {
        "name": "content",
        "type": "text"
    },
    {
        "name": "propositions",
        "type": "text"
    },
    {
        "name": "label",
        "type": "tag",
        "attrs": {
            "sortable": True
        }
    },
    {
        "name": "text_embedding",
        "type": "vector",
        "attrs": {
            "dims": hf.dims,
            "distance_metric": "cosine",
            "algorithm": "hnsw",
            "datatype": "float32"
        }
    }
  ]
})

# connect to redis
client = Redis.from_url(REDIS_URL)

# create an index
index = SearchIndex(schema, client)
index.create(overwrite=True, drop=True)

In [ ]:
# use the CLI to see the created index
!rvl index listall

In [ ]:
!rvl index info -i redisvl

### Process and load data using RedisVL
Below we use the RedisVL index to simply load the list of document chunks to Redis db.

In [ ]:
# load expects an iterable of dictionaries
from redisvl.redis.utils import array_to_buffer

data = [
    {
        'label': f'ID-{i}',
        'content': chunk.page_content,
        'propositions': " ".join(props[i]),
        # For HASH -- must convert embeddings to bytes
        'text_embedding': array_to_buffer(embeddings[i])
    } for i, chunk in enumerate(chunks)
]

# RedisVL handles batching automatically
keys = index.load(data)

### Query the database
Now we can use the RedisVL index to perform similarity search operations with Redis

In [ ]:
from redisvl.query import VectorQuery

query = "Nike profit margins and company performance"

query_embedding = hf.embed(query)

vector_query = VectorQuery(
    vector=query_embedding,
    vector_field_name="text_embedding",
    num_results=3,
    return_fields=["label", "propositions"],
    return_score=True
)

# show the raw redis query
str(vector_query)

In [ ]:
# execute the query with RedisVL
index.query(vector_query)

In [ ]:
# paginate through results
for result in index.paginate(vector_query, page_size=1):
    print(result[0]["label"], result[0]["vector_distance"], flush=True)

### Sort by alternative fields

In [ ]:
# Sort by label field after vector search limits to topK
vector_query = VectorQuery(
    vector=query_embedding,
    vector_field_name="text_embedding",
    num_results=4,
    return_fields=["label"],
    return_score=True
)

# Decompose vector_query into the core query and the params
query = vector_query.query
params = vector_query.params

# Pass query and params direct to index.search()
result = index.search(
    query.sort_by("label", asc=True),
    params
  )

[doc.__dict__ for doc in result.docs]

### Add filters to vector queries

In [ ]:
from redisvl.query.filter import Text

vector_query = VectorQuery(
    vector=query_embedding,
    vector_field_name="text_embedding",
    num_results=4,
    return_fields=["propositions"],
    return_score=True
)

# Set a text filter
text_filter = Text("content") % "profit"

vector_query.set_filter(text_filter)

index.query(vector_query)

### Range queries in RedisVL

In [ ]:
from redisvl.query import RangeQuery

range_query = RangeQuery(
    vector=query_embedding,
    vector_field_name="text_embedding",
    num_results=4,
    return_fields=["propositions"],
    return_score=True,
    distance_threshold=0.5  # find all items with a semantic distance of less than 0.5
)

In [ ]:
index.query(range_query)

In [ ]:
# Add filter to range query
range_query.set_filter(text_filter)

index.query(range_query)

## Building a RAG Pipeline with RedisVL

We're going to build a complete RAG pipeline from scratch incorporating the following components:

- Standard retrieval and chat completion
- Query re-writing to improve accuracy
- Semantic caching to improve performance
- Conversational session history to improve personalization

### Setup RedisVL AsyncSearchIndex

In [ ]:
from redis.asyncio import Redis
from redisvl.index import AsyncSearchIndex

client = Redis.from_url(REDIS_URL)
index = AsyncSearchIndex(index.schema, client)

### Define OpenAI RAG Helpers & Prompts

In [ ]:

async def answer_question(index: AsyncSearchIndex, query: str):
    """Answer the user's question"""

    SYSTEM_PROMPT = """You are a helpful financial analyst assistant that has access
    to public financial 10k documents in order to answer users questions about company
    performance, ethics, characteristics, and core information.
    """

    query_vector = hf.embed(query)
    # Fetch context from Redis using vector search
    context = await retrieve_context(index, query_vector)
    # Generate contextualized prompt and feed to OpenAI
    response = await openai.AsyncClient().chat.completions.create(
        model=CHAT_MODEL,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": promptify(query, context)}
        ],
        temperature=0.1,
        seed=42
    )
    # Response provided by LLM
    return response.choices[0].message.content


async def retrieve_context(index: AsyncSearchIndex, query_vector) -> str:
    """Fetch the relevant context from Redis using vector search"""
    results = await index.query(
        VectorQuery(
            vector=query_vector,
            vector_field_name="text_embedding",
            return_fields=["propositions"],
            num_results=3
        )
    )
    content = "\n".join([result["propositions"] for result in results])
    return content


def promptify(query: str, context: str) -> str:
    return f'''Use the provided context below derived from public financial
    documents to answer the user's question. If you can't answer the user's
    question, based on the context; do not guess. If there is no context at all,
    respond with "I don't know".

    User question:

    {query}

    Helpful context:

    {context}

    Answer:
    '''

### Vanilla Async RAG

In [ ]:
# Generate a list of questions
questions = [
    "What is the trend in the company's revenue and profit over the past few years?",
    "What are the company's primary revenue sources?",
    "How much debt does the company have, and what are its capital expenditure plans?",
    "What does the company say about its environmental, social, and governance (ESG) practices?",
    "What is the company's strategy for growth?"
]

In [ ]:
import asyncio

results = await asyncio.gather(*[
    answer_question(index, question) for question in questions
])

In [ ]:
import pandas as pd

pd.DataFrame(columns=["question", "answer"], data=list(zip(questions, results)))

### Improve accuracy with query rewriting / expansion

We can also use the power on an LLM to rewrite or expand an input question.

Example: https://github.com/langchain-ai/langchain/blob/master/templates/rewrite-retrieve-read/rewrite_retrieve_read/chain.py

In [ ]:
# An example question that is a bit simplistic...
await answer_question(index, "How big is the company?")

In [ ]:
async def rewrite_query(query: str):
    """Rewrite the user's original query"""

    SYSTEM_PROMPT = """Given the user's input question below, find a better or
    more complete way to phrase this question in order to improve semantic search
    engine retrieval quality over a set of SEC 10K PDF docs. Return the rephrased
    question as a string in a JSON response under the key "query"."""

    response = await openai.AsyncClient().chat.completions.create(
        model=CHAT_MODEL,
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"Original input question from user: {query}"}
        ],
        temperature=0.1,
        seed=42
    )
    # Response provided by LLM
    return json.loads(response.choices[0].message.content)["query"]

In [ ]:
# Example Sinple Query Rewritten
await rewrite_query("How big is the company?")

In [ ]:
async def answer_question(index: AsyncSearchIndex, query: str):
    """Answer the user's question"""

    SYSTEM_PROMPT = """You are a helpful financial analyst assistant that has access
    to public financial 10k documents in order to answer users questions about company
    performance, ethics, characteristics, and core information.
    """

    # Rewrite the query using an LLM
    rewritten_query = await rewrite_query(query)
    query_vector = hf.embed(rewritten_query)
    # Fetch context from Redis using vector search
    context = await retrieve_context(index, query_vector)
    # Generate contextualized prompt and feed to OpenAI
    response = await openai.AsyncClient().chat.completions.create(
        model=CHAT_MODEL,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": promptify(rewritten_query, context)}
        ],
        temperature=0.1,
        seed=42
    )
    # Response provided by LLM
    return response.choices[0].message.content

In [ ]:
# Now try again with query re-writing enabled
await answer_question(index, "How big is the company?")

### Improve performance and cut costs with LLM caching

In [ ]:
from redisvl.extensions.llmcache import SemanticCache

llmcache = SemanticCache(
    name="llmcache",
    vectorizer=hf,
    redis_url=REDIS_URL,
    ttl=120,
    distance_threshold=0.2
)

In [ ]:
from functools import wraps

# Create an LLM caching decorator
def cache(func):
    @wraps(func)
    async def wrapper(index, query_text, *args, **kwargs):
        query_vector = llmcache._vectorizer.embed(query_text)

        # Check the cache with the vector
        if result := llmcache.check(vector=query_vector):
            return result[0]['response']

        response = await func(index, query_text, query_vector=query_vector)
        llmcache.store(query_text, response, query_vector)
        return response
    return wrapper


@cache
async def answer_question(index: AsyncSearchIndex, query: str, **kwargs):
    """Answer the user's question"""

    SYSTEM_PROMPT = """You are a helpful financial analyst assistant that has access
    to public financial 10k documents in order to answer users questions about company
    performance, ethics, characteristics, and core information.
    """

    context = await retrieve_context(index, kwargs["query_vector"])
    response = await openai.AsyncClient().chat.completions.create(
        model=CHAT_MODEL,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": promptify(query, context)}
        ],
        temperature=0.1,
        seed=42
    )
    # Response provided by GPT-3.5
    return response.choices[0].message.content

In [ ]:
query = "What was Nike's revenue last year compared to this year??"

await answer_question(index, query)

In [ ]:
query = "What was Nike's total revenue in the last year compared to now??"

await answer_question(index, query)

# notice no HTTP request to OpenAI since this question is "close enough" to the last one

### Offload session history to Redis

In order to preserve state in the conversation, it's imperitive to offload conversation history to a database that can handle high transaction throughput for writes/reads to limit system latency.

We can store message history for a particular user session in a Redis List data type.


In [ ]:
import json


async def get_messages(index: AsyncSearchIndex, user_id: str) -> list:
    """Get all messages associated with a session"""
    return [
        json.loads(msg) for msg in await index.client.lrange(f"messages:{user_id}", 0, -1)
    ]

async def add_messages(index: AsyncSearchIndex, user_id: str, messages: list):
    """Add chat messages to a Redis List"""
    return await index.client.rpush(
        f"messages:{user_id}", *[json.dumps(msg) for msg in messages]
    )

async def clear_history(index: AsyncSearchIndex, user_id: str):
    """Clear session chat"""
    await index.client.delete(f"messages:{user_id}")

async def answer_question(index: AsyncSearchIndex, query: str):
    """Answer the user's question with historical context and caching baked-in"""

    SYSTEM_PROMPT = """You are a helpful financial analyst assistant that has access
    to public financial 10k documents in order to answer users questions about company
    performance, ethics, characteristics, and core information.
    """

    query_vector = llmcache._vectorizer.embed(query)

    # Check the cache with the vector
    if result := llmcache.check(vector=query_vector):
        answer = result[0]['response']
    else:
        context = await retrieve_context(index, query_vector)
        session = await get_messages(index, "tyler")
        messages = (
            [{"role": "system", "content": SYSTEM_PROMPT}] +
            session +
            [{"role": "user", "content": promptify(query, context)}]
        )
        # Response provided by GPT-4
        response = await openai.AsyncClient().chat.completions.create(
            model=CHAT_MODEL,
            messages=messages,
            temperature=0.1,
            seed=42
        )
        answer = response.choices[0].message.content

    # Add message history
    await add_messages(index, "tyler", [
        {"role": "user", "content": query},
        {"role": "assistant", "content": answer}
    ])
    return answer

In [ ]:
# Setup Session
await clear_history(index, "tyler")

# Simple Chat
while True:
    query = input()
    if query.lower() == "exit":
        break
    answer = await answer_question(index, query)
    print(answer, flush=True)


In [ ]:
await index.client.lrange("messages:tyler", 0, -1)

## Your Next Steps

While a good start, there is still more to do. **For example**:
- we could utilize message history to generate an updated and contextualized query to use for retrieval and answer generation (with an LLM). Otherwise, there can be a disconnect between what a user is asking (in context) and what they are asking in isolation.
- we could utilize an LLM to summarize conversation history to use as context instead of passing the whole slew of messages to the Chat endpoint.
- we could utilize semantic properties of the message history (or summaries) in order to fetch only relevant conversation bits (vector search).
- we could utilize a technique like HyDE ( a form of query rewriting ) to improve the retrieval quality from raw user input to source documents OR try to break down user questions into sub questions and fetch / join context based on the different searces.
- we could incorporate semantic routing to take a broken down question and route to different data sources, indices, or query types (etc).
- we could add semantic guardrails on the front end or back end of the conversation I/O to ensure we are within bounds of approved topics.

## Cleanup

Clean up the database.

In [ ]:
await index.client.flushall()

Now that you have tried the easy-to-use RedisVL client, try your hand with LangChain -- the highest level of abstraction for using and integrating Redis as a vector database.


<a href="https://colab.research.google.com/github/Redislabs-Solution-Architects/financial-vss/blob/main/langchain-03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>